# Testing Ray Setup for LLM Serving

In [1]:
from snowflakeray.cluster_init import SnowflakeRay
from snowflake.snowpark import Session
import json
from pprint import pprint

### Setup Snowpark Session

In [2]:
with open('creds.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']
    account = data["account"]
    warehouse = data["warehouse"]
    database = data["database"]
    schema = data["schema"]
    role = data["role"]


CONNECTION_PARAMETERS = {
    'account': account,
    'user': username,
    'password': password,
    'schema': schema,
    'database': database,
    'warehouse': warehouse,
    'role': role,
    "session_parameters": {"PYTHON_CONNECTOR_QUERY_RESULT_FORMAT": "json"}
}

In [3]:
session = Session.builder.configs(CONNECTION_PARAMETERS).create()

### Setup Prerequisites
##### Step 0: Run the code in setup-ray-prerequisites.sql

In [4]:
project_name = "llm serving"

### Initiate SnowflakeRay object (no default pool names selected)

In [5]:
snowflake_ray = SnowflakeRay(session=session, project_name=project_name)

##### Note that if you are signed up for Block Storage PrPr, add the parameter need_block_storage_for_ray_logs=True below

In [7]:
endpoints = snowflake_ray.setup_ray_cluster(compute_pool_type="GPU", num_worker_nodes=4, requested_num_head_gpus=1, requested_num_worker_gpus=1, stage_name_for_specs="RAY_SPECS", stage_name_for_artifacts="ARTIFACTS", external_access_integrations=["ALLOW_ALL_EAI"], 
                              ray_requirements=["ray[data]==2.9.3", "ray[client]==2.9.3", "ray[default]==2.9.3", "ray[serve]==2.9.3"],
                              pip_requirements=["jupyterlab", "py-spy", "ipywidgets", "virtualenv", "starlette<=0.34.0", "datasets", "numpy", "transformers", "torch==2.1.2", "evaluate", "accelerate", "tokenizers", "pandas==1.5.3", "vllm==0.2.7"])

ray_head_precreated_compute_pool: False
ray_worker_precreated_compute_pool: False


INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Created image repo: sfsenorthamerica-demo391.registry.snowflakecomputing.com/ray_db/ray_schema/spcs_ray_image_repollmserving
INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Similar environment detected. Using existing image sfsenorthamerica-demo391.registry.snowflakecomputing.com/ray_db/ray_schema/spcs_ray_image_repollmserving/ray_grafana:57d8671a2da19fcd575e7be9710d553b5cc817ca to skip image build. To disable this feature, set 'force_image_build=True' in deployment options
INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Similar environment detected. Using existing image sfsenorthamerica-demo391.registry.snowflakecomputing.com/ray_db/ray_schema/spcs_ray_image_repollmserving/ray_prometheus:ae2d4ca55b4de0729a82ff0e88903d705be48b5b to skip image build. To disable this feature, set 'force_image_build=True' in deployment options
INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Building the Docker image an

In [8]:
endpoints

[{'api': 'biaoemj-sfsenorthamerica-demo391.snowflakecomputing.app'},
 {'notebook': 'biaoemn-sfsenorthamerica-demo391.snowflakecomputing.app'},
 {'prometheus': 'biaoemr-sfsenorthamerica-demo391.snowflakecomputing.app'},
 {'grafana': 'biaoemv-sfsenorthamerica-demo391.snowflakecomputing.app'},
 {'ray-dashboard': 'biaoemz-sfsenorthamerica-demo391.snowflakecomputing.app'}]

### Get service endpoints independent of the command above (optional)

In [9]:
pprint(snowflake_ray.get_public_endpoints())

[{'api': 'biaoemj-sfsenorthamerica-demo391.snowflakecomputing.app'},
 {'notebook': 'biaoemn-sfsenorthamerica-demo391.snowflakecomputing.app'},
 {'prometheus': 'biaoemr-sfsenorthamerica-demo391.snowflakecomputing.app'},
 {'grafana': 'biaoemv-sfsenorthamerica-demo391.snowflakecomputing.app'},
 {'ray-dashboard': 'biaoemz-sfsenorthamerica-demo391.snowflakecomputing.app'}]


### Get ray head service status

In [10]:
pprint(snowflake_ray.get_ray_head_service_status())

[{'containerName': 'head',
  'image': 'sfsenorthamerica-demo391.registry.snowflakecomputing.com/ray_db/ray_schema/spcs_ray_image_repollmserving/ray_head:479260125434936c6990e38c4f23e9247d80728b',
  'instanceId': '0',
  'message': 'Running',
  'restartCount': 0,
  'serviceName': 'SPCSRAYHEADSERVICELLMSERVING',
  'startTime': '2024-04-08T22:59:55Z',
  'status': 'READY'},
 {'containerName': 'prometheus',
  'image': 'sfsenorthamerica-demo391.registry.snowflakecomputing.com/ray_db/ray_schema/spcs_ray_image_repollmserving/ray_prometheus:ae2d4ca55b4de0729a82ff0e88903d705be48b5b',
  'instanceId': '0',
  'message': 'Running',
  'restartCount': 0,
  'serviceName': 'SPCSRAYHEADSERVICELLMSERVING',
  'startTime': '2024-04-08T23:01:49Z',
  'status': 'READY'},
 {'containerName': 'grafana',
  'image': 'sfsenorthamerica-demo391.registry.snowflakecomputing.com/ray_db/ray_schema/spcs_ray_image_repollmserving/ray_grafana:57d8671a2da19fcd575e7be9710d553b5cc817ca',
  'instanceId': '0',
  'message': 'Running

### Get ray head service logs

In [11]:
pprint(snowflake_ray.get_ray_head_logs())

('[{\'SYSTEM$GET_SERVICE_LOGS\': "+ WORKLOAD=rayhead\\n++ ifconfig eth0\\n++ '
 "sed -En -e 's/.*inet ([0-9.]+).*/\\\\1/p'\\n+ eth0Ip=10.244.16.8\\n+ echo "
 "'WORKLOAD: rayhead'\\nWORKLOAD: rayhead\\n+ '[' rayhead == rayhead ']'\\n+ "
 "'[' -z http://SPCSRAYHEADSERVICELLMSERVING:3000 ']'\\n+ '[' -z "
 "http://SPCSRAYHEADSERVICELLMSERVING:9090 ']'\\n+ export "
 'RAY_GRAFANA_HOST=http://SPCSRAYHEADSERVICELLMSERVING:3000\\n+ '
 'RAY_GRAFANA_HOST=http://SPCSRAYHEADSERVICELLMSERVING:3000\\n+ export '
 'RAY_PROMETHEUS_HOST=http://SPCSRAYHEADSERVICELLMSERVING:9090\\n+ '
 'RAY_PROMETHEUS_HOST=http://SPCSRAYHEADSERVICELLMSERVING:9090\\n+ export '
 'log_dir=/raylogs/ray\\n+ log_dir=/raylogs/ray\\nMaking log directory '
 "/raylogs/ray...\\n+ echo 'Making log directory /raylogs/ray...'\\n+ mkdir -p "
 '/raylogs/ray\\n+ export RAY_ENABLE_RECORD_ACTOR_TASK_LOGGING=1\\n+ '
 'RAY_ENABLE_RECORD_ACTOR_TASK_LOGGING=1\\n+ jupyter lab '
 '--generate-config\\nWriting default config to: '
 '/root/.jupyter/j

### Get ray worker service status

In [13]:
pprint(snowflake_ray.get_ray_worker_service_status())

[{'containerName': 'worker',
  'image': 'sfsenorthamerica-demo391.registry.snowflakecomputing.com/ray_db/ray_schema/spcs_ray_image_repollmserving/ray_worker:da9a87721f156d630234ee83e598f97dc0befae4',
  'instanceId': '0',
  'message': 'Running',
  'restartCount': 0,
  'serviceName': 'SPCSRAYWORKERSERVICELLMSERVING',
  'startTime': '2024-04-08T23:00:18Z',
  'status': 'READY'},
 {'containerName': 'worker',
  'image': 'sfsenorthamerica-demo391.registry.snowflakecomputing.com/ray_db/ray_schema/spcs_ray_image_repollmserving/ray_worker:da9a87721f156d630234ee83e598f97dc0befae4',
  'instanceId': '1',
  'message': 'Running',
  'restartCount': 0,
  'serviceName': 'SPCSRAYWORKERSERVICELLMSERVING',
  'startTime': '2024-04-08T23:00:27Z',
  'status': 'READY'},
 {'containerName': 'worker',
  'image': 'sfsenorthamerica-demo391.registry.snowflakecomputing.com/ray_db/ray_schema/spcs_ray_image_repollmserving/ray_worker:da9a87721f156d630234ee83e598f97dc0befae4',
  'instanceId': '2',
  'message': 'Running',

### Get ray worker logs

In [14]:
pprint(snowflake_ray.get_ray_worker_logs())

('[{\'SYSTEM$GET_SERVICE_LOGS\': "+ WORKLOAD=rayworker\\n++ ifconfig eth0\\n++ '
 "sed -En -e 's/.*inet ([0-9.]+).*/\\\\1/p'\\n+ eth0Ip=10.244.15.8\\n+ echo "
 "'WORKLOAD: rayworker'\\nWORKLOAD: rayworker\\n+ '[' rayworker == rayhead "
 "']'\\n+ '[' rayworker == rayworker ']'\\n+ '[' -z "
 "SPCSRAYHEADSERVICELLMSERVING:6379 ']'\\n+ ray start "
 '--node-ip-address=10.244.15.8 --disable-usage-stats '
 '--address=SPCSRAYHEADSERVICELLMSERVING:6379 --num-cpus=8 '
 '--object-manager-port=8076 --node-manager-port=8077 '
 '--runtime-env-agent-port=8078 --dashboard-agent-grpc-port=8079 '
 '--dashboard-agent-listen-port=8081 --metrics-export-port=8082 '
 '--min-worker-port=10002 --max-worker-port=10021 --block\\n2024-04-08 '
 '23:00:19,232\\tINFO scripts.py:926 -- \\x1bLocal node IP\\x1b: '
 '\\x1b10.244.15.8\\x1b\\n[2024-04-08 23:01:58,649 W 10 10] '
 'global_state_accessor.cc:465: Some processes that the driver needs to '
 "connect to have not registered with GCS, so retrying. Have you run 'ra

### Delete all services

In [ ]:
#snowflake_ray.delete_all_services()

INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Deleted service: SPCSRAYHEADSERVICELLMSERVING
INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Deleted service: SPCSRAYWORKERSERVICELLMSERVING


### Suspend all compute pools

In [10]:
#snowflake_ray.suspend_all_compute_pools()

INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Suspended compute pool: RAY_HEAD_CP_LLM_SERVING
INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Suspended compute pool: RAY_WORKER_CP_LLM_SERVING


### Delete all compute pools

In [15]:
#snowflake_ray.delete_all_compute_pools()

INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Deleted service: SPCSRAYHEADSERVICELLMSERVING
INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Deleted service: SPCSRAYWORKERSERVICELLMSERVING
INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Deleted compute pool: RAY_HEAD_CP_RAYDBLLMSERVING
INFO:snowflakeray.deploy_client.utils.cluster_init_helper:Deleted compute pool: RAY_WORKER_CP_RAYDBLLMSERVING


### Close snowpark session

In [16]:
#session.close()